Import statements

In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import linear_model, preprocessing
import os
import glob
import pandas as pd
import nltk
from nltk import tokenize
from nltk.stem import WordNetLemmatizer
import re 
# nltk.download('stopwords')
# nltk.download()
from nltk.corpus import stopwords 

In [3]:
nltk.download('punkt')
nltk.download('all')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora

True

Compile and run from here.

To load the train and validation data from TSV files.

Arguments: train file, validation file

Returns:

tr_st: Training sentences
tr_ft: Training Future Labels
tr_pt: Trainiing past labels
val_st: Validation sentences
val_ft: Validation Future Labels
val_pt: Validation past labels

In [0]:
def load_train_val(train,val):
  df= pd.read_csv(train,delimiter='\t',encoding='utf-8')
  tr_st=np.array(df['sentence'].values)
  tr_ft=np.array(df['Future'].values)
  tr_pt=np.array(df['Past'].values)
  df= pd.read_csv(val,delimiter='\t',encoding='utf-8')
  val_st=np.array(df['sentence'].values)
  val_ft=np.array(df['Future'].values)
  val_pt=np.array(df['Past'].values)
  return tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt


Method to load the test data from TSV file.

Arguments: test file

Returns:

ts_st: Test sentences
ts_ft: Test Future Labels
ts_pt: Test past labels

In [0]:
def load_test(test):
  df= pd.read_csv(test,delimiter='\t',encoding='utf-8')
  ts_st=np.array(df['sentence'].values)
  ts_ft=np.array(df['Future'].values)
  ts_pt=np.array(df['Past'].values)
  return ts_st,ts_ft,ts_pt

To preprocess sentences:
Removing some special characters, spaces and tabs, and conversion to lower case

In [0]:
def preprocess_sentences(X):
  documents = []
  stemmer = WordNetLemmatizer()
  for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'^"', '', document)
    document = re.sub(r'"$', '', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    #     document = ' '.join(document)
    documents.append(document)
  print(len(documents))
  return documents

Converts sentences to lists of features

Uncomment code as needed for features



In [0]:
def sent_to_features(documents):
  feature_docs=[]
  for document in documents:
    tokens=nltk.word_tokenize(document)
    bigrams = list(nltk.bigrams(tokens))
    #pos_bigrams = list(nltk.bigrams([pos for (word,pos) in nltk.pos_tag(tokens)]))
    document=[word for (word,pos) in nltk.pos_tag(tokens)]#+' '+pos
#     document.extend([w1+'_'+w2 for (w1,w2) in bigrams])
    #     document.extend([w1+'_'+w2 for (w1,w2) in pos_bigrams])
    #     document.extend([pos for (word,pos) in nltk.pos_tag(tokens)])
    #     document = document.split()
    #     document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    feature_docs.append(document)
  return feature_docs

This is not necesasry unless the code is being run on a Colab notebook, and the data is stored on Google Drive

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Path to the location where the data resides

In [0]:
datapath = 'gdrive/My Drive/data/'

Loading the train and validation data

Preprocessing the data

Feature extraction if necessary

In [10]:
#loading train and validation data
trainfile=datapath+'train.txt'
valfile=datapath+'validation.txt'
tr_st,tr_ft,tr_pt, val_st,val_ft,val_pt=load_train_val(trainfile,valfile)

#preprocessing
tr_st = preprocess_sentences(tr_st)
val_st = preprocess_sentences(val_st)

#feature extraction
tr_st = sent_to_features(tr_st)
val_st = sent_to_features(val_st)

print('Train size: ',len(tr_st))
print('Val size: ',len(val_st))

1776
592
Train size:  1776
Val size:  592


In [14]:
print(tr_st[1])

friends and adversaries abroad were asking whether america had lost its nerve


Imports necessary for LSTM 

In [0]:
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from keras import optimizers

# from tensorflow.keras.backend import set_session
import keras.backend as K
import nltk
import string
import numpy as np
import pandas as pd
import re
from tqdm import tqdm


Creating a Keras Tokenizer and fitting it on train and validation data

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))

In [0]:
from gensim.models import Word2Vec

pos_model = Word2Vec.load(datapath+'model_300d.bin')
# print(pos_model)
words = list(pos_model.wv.vocab)
# print(words)
# print(len(words))
# print(pos_model['NN'])



Initializing vocabulary size

In [0]:
vocabulary_size = len(tokenizer.word_index)+1
# print(tokenizer.word_counts)
# print(tokenizer.document_count)
# print(tokenizer.word_index)
# print(tokenizer.word_docs)


In [14]:
print(vocabulary_size)

6744


In [0]:
# #feature extraction
# tr_st = sent_to_features(tr_st)
# val_st = sent_to_features(val_st)

Extracting sequences from train and validation data, and padding them to a maximum length

In [15]:

# vocabulary_size = 7000
# tokenizer.fit_on_texts(np.concatenate((tr_st,val_st)))

sequences = tokenizer.texts_to_sequences(tr_st)
tr_data = pad_sequences(sequences, maxlen=145)
sequences = tokenizer.texts_to_sequences(val_st)
val_data = pad_sequences(sequences, maxlen=145)

print(len(tr_data))
print(len(val_data))

1776
592


Loading Glove embeddings

In [16]:
f = open(datapath+'glove.6B.300d.txt')
embedding_values = {}
for line in tqdm(f):
  value = line.split(' ')
  word = value[0]
  coef = np.array(value[1:],dtype = 'float32')
  embedding_values[word]=coef

400000it [00:36, 10928.29it/s]


Adding embedding values of POS tags

In [0]:
for word in words:
  coef=np.array(pos_model[word],dtype = 'float32')
  embedding_values[word]=coef

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [17]:
len(embedding_values)

400000

Creating an embedding matrix for words 

In [18]:
embedding_matrix = np.zeros((vocabulary_size,300))
for word,i in tqdm(tokenizer.word_index.items()):
  values = embedding_values.get(word)
  if values is not None:
    embedding_matrix[i] = values

100%|██████████| 6743/6743 [00:00<00:00, 272090.66it/s]


In [0]:
import numpy as np
import tensorflow as tf
import random as rn
from numpy.random import seed
from tensorflow import set_random_seed


seed(2)
set_random_seed(2)
rn.seed(2)
os.environ['PYTHONHASHSEED'] = '0'
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


In [0]:
def session_refresh():
  seed(2)
  set_random_seed(2)
  rn.seed(2)
  os.environ['PYTHONHASHSEED'] = '0'
  session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
  K.set_session(sess)
  print('refreshed session')

BiLSTM for Future task

In [29]:
#Future
# K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2)))

model.add(Dense(64,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

adg= optimizers.Adagrad(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

filepath=datapath+'/models/'+"future-1-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.summary()
model.fit(tr_data, tr_ft, validation_data=(val_data,val_ft), epochs=10,callbacks=callbacks_list,batch_size=64)
model_json = model.to_json()
with open(datapath+"future-1-model.json", "w") as json_file:
    json_file.write(model_json)
# model.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_ft,val_ft)), epochs=10,batch_size=64)#  validation_data=(val_data,val_ft),

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 145, 300)          2023200   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 290)               517360    
_________________________________________________________________
dense_11 (Dense)             (None, 64)                18624     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 2,559,249
Trainable params: 536,049
Non-trainable params: 2,023,200
_________________________________________________________________
Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 17s 10ms/step - loss: 0.7113 - acc: 0.6120 - val_loss: 0.5403 - val_acc: 0.7297

Epoch 00001: val_acc improved from -inf to 0.72973, saving model to gdrive

Stacked BiLSTM for Future task

In [45]:
#Future LSTM2
# K.clear_session()
K.set_session(sess)
model2 = Sequential()
model2.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)

filepath=datapath+'/models/'+"future-2-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 145, 300)          2023200   
_________________________________________________________________
bidirectional_19 (Bidirectio (None, 145, 290)          517360    
_________________________________________________________________
bidirectional_20 (Bidirectio (None, 290)               505760    
_________________________________________________________________
dense_25 (Dense)             (None, 64)                18624     
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 65        
Total params: 3,065,009
Trainable params: 1,041,809
Non-trainable params: 2,023,200
_________________________________________________________________


In [38]:
model2.fit(tr_data, tr_ft, validation_data=(val_data,val_ft), epochs=10,callbacks=callbacks_list,batch_size=64)  #epoch 15 batch 128 no relu hidden   epoch 7 batch 128 w/ relu hidden
model_json = model.to_json()
with open(datapath+"future-2-model.json", "w") as json_file:
    json_file.write(model_json)

Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 38s 21ms/step - loss: 0.7058 - acc: 0.5822 - val_loss: 0.5670 - val_acc: 0.7416

Epoch 00001: val_acc improved from -inf to 0.74155, saving model to gdrive/My Drive/data//models/future-2-best.hdf5
Epoch 2/10
1776/1776 [==============================] - 29s 16ms/step - loss: 0.5436 - acc: 0.7461 - val_loss: 0.4646 - val_acc: 0.7939

Epoch 00002: val_acc improved from 0.74155 to 0.79392, saving model to gdrive/My Drive/data//models/future-2-best.hdf5
Epoch 3/10
1776/1776 [==============================] - 30s 17ms/step - loss: 0.4981 - acc: 0.7866 - val_loss: 0.4459 - val_acc: 0.7939

Epoch 00003: val_acc did not improve from 0.79392
Epoch 4/10
1776/1776 [==============================] - 30s 17ms/step - loss: 0.4542 - acc: 0.7967 - val_loss: 0.4508 - val_acc: 0.8007

Epoch 00004: val_acc improved from 0.79392 to 0.80068, saving model to gdrive/My Drive/data//models/future-2-best.hdf5
E

Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 62s 35ms/step - loss: 0.6709 - acc: 0.6267 - val_loss: 0.5295 - val_acc: 0.7686
Epoch 2/10
1776/1776 [==============================] - 56s 32ms/step - loss: 0.5727 - acc: 0.7230 - val_loss: 0.5095 - val_acc: 0.7618
Epoch 3/10
1776/1776 [==============================] - 57s 32ms/step - loss: 0.5068 - acc: 0.7742 - val_loss: 0.4501 - val_acc: 0.7872
Epoch 4/10
1776/1776 [==============================] - 57s 32ms/step - loss: 0.4625 - acc: 0.7922 - val_loss: 0.4338 - val_acc: 0.8074
Epoch 5/10
1776/1776 [==============================] - 58s 32ms/step - loss: 0.4359 - acc: 0.8012 - val_loss: 0.4197 - val_acc: 0.8057
Epoch 6/10
1776/1776 [==============================] - 57s 32ms/step - loss: 0.3968 - acc: 0.8294 - val_loss: 0.4183 - val_acc: 0.8243
Epoch 7/10
1776/1776 [==============================] - 58s 32ms/step - loss: 0.3847 - acc: 0.8249 - val_loss: 0.4313 - val_acc: 0.8108
Epoch 8/10
1776/1776 [==============================] - 58s 32ms/step - loss: 0.3585 - acc: 0.8457 - val_loss: 0.4272 - val_acc: 0.8091
Epoch 9/10
1776/1776 [==============================] - 56s 32ms/step - loss: 0.3308 - acc: 0.8581 - val_loss: 0.4347 - val_acc: 0.8091
Epoch 10/10
1776/1776 [==============================] - 56s 32ms/step - loss: 0.3063 - acc: 0.8632 - val_loss: 0.4721 - val_acc: 0.7787

Training on whole training data after parameter tuning, in order to use the model to evaluate the test data

In [47]:
model2.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_ft,val_ft)), epochs=3,batch_size=64)

model_json = model2.to_json()
with open(datapath+'/models/'+"future-2-full-model.json", "w") as json_file:
    json_file.write(model_json)
model2.save_weights(datapath+'/models/'+"future-2-full-best.hdf5")
print("Saved model to disk")

Epoch 1/3
2368/2368 [==============================] - 37s 16ms/step - loss: 0.3892 - acc: 0.8247
Epoch 2/3
2368/2368 [==============================] - 36s 15ms/step - loss: 0.3542 - acc: 0.8429
Epoch 3/3
2368/2368 [==============================] - 36s 15ms/step - loss: 0.3433 - acc: 0.8514
Saved model to disk


Loading the Test data

preprocessing the data

Extracting sequences

Padding the sequences



In [48]:
testfile=datapath+'test.txt'
ts_st,ts_ft,ts_pt=load_test(testfile)

#preprocessing
ts_st = preprocess_sentences(ts_st)

#feature extraction
#ts_st = sent_to_features(ts_st)
sequences = tokenizer.texts_to_sequences(ts_st)
ts_data = pad_sequences(sequences, maxlen=145)
print(ts_data.shape)
print(ts_ft.shape)
# test_loss, test_acc = model.evaluate(np.append((ts_data,ts_ft),1))
# print('Test Loss: {}'.format(test_loss))
# print('Test Accuracy: {}'.format(test_acc))


# print('Accuaracy = {0:f}'.format(acc))

593
(593, 145)
(593,)


Evaluating the test data for future task 

In [49]:

scores = model2.evaluate(ts_data, ts_ft, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))


Accuracy: 82.80%


Future Test Accuracy: 82.80% ; epochs 5+3, model: BiLSTM + Dense



In [74]:
# testf='sotu_2019.tsv'
# df= pd.read_csv(testf,delimiter='\t',encoding='utf-8')
# full_st=np.array(df['sentence'].values)
# full_st = preprocess_sentences(full_st)

# #feature extraction
# #ts_st = sent_to_features(ts_st)
# sequences = tokenizer.texts_to_sequences(full_st)
# full_st_data = pad_sequences(sequences, maxlen=145)


# y_pred = model2.predict(full_st_data, batch_size=64, verbose=1)
# y_pred =(y_pred>0.5)




391/391 [==============================] - 4s 9ms/step


In [0]:
# df['ft']=list(map(int,y_pred))
# df.to_csv('sotu_future.tsv',sep='\t')

In [0]:
# serialize model to JSON
# model_json = model2.to_json()
# with open(datapath+"future_model2.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model2.save_weights(datapath+"future_model2.h5")
# print("Saved model to disk")
 
# # later...
 
# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")

Generating a classification report for the test data for future task

In [50]:
from sklearn.metrics import classification_report
y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_ft, y_pred))

593/593 [==============================] - 7s 12ms/step
              precision    recall  f1-score   support

         0.0       0.87      0.77      0.82       293
         1.0       0.80      0.88      0.84       300

   micro avg       0.83      0.83      0.83       593
   macro avg       0.83      0.83      0.83       593
weighted avg       0.83      0.83      0.83       593





Future Test Accuracy: 82.80% ; epochs 5+3, model: BiLSTM + Dense

precision    recall  f1-score   support

         0.0       0.87      0.77      0.82       293
         1.0       0.80      0.88      0.84       300

   micro avg       0.83      0.83      0.83       593
   macro avg       0.83      0.83      0.83       593
weighted avg       0.83      0.83      0.83       593


In [0]:
# scores = model2.evaluate(ts_data, ts_ft, verbose=0)

# print("Accuracy: %.2f%%" % (scores[1]*100))
# y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
# y_pred =(y_pred>0.5)

# print(classification_report(ts_ft, y_pred))


In [0]:
K.clear_session()

BiLSTM model for Past task

In [57]:
#Past

# K.clear_session()
K.set_session(sess)
model = Sequential()
model.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)
model.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

filepath=datapath+'/models/'+"past-1-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()
model.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=10,callbacks=callbacks_list,batch_size=64) #eposh 8 batch 128 no relu hidden
model_json = model.to_json()
with open(datapath+"past-1-model.json", "w") as json_file:
    json_file.write(model_json)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 145, 300)          2023200   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 290)               517360    
_________________________________________________________________
dense_7 (Dense)              (None, 64)                18624     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 2,559,249
Trainable params: 536,049
Non-trainable params: 2,023,200
_________________________________________________________________
Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 18s 10ms/step - loss: 0.6555 - acc: 0.6329 - val_loss: 0.5829 - val_acc: 0.7162

Epoch 00001: val_acc improved from -inf to 0.71622, saving model to gdrive

In [63]:

# K.clear_session()
session_refresh()
model = Sequential()
model.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

filepath=datapath+'/models/'+"past-1-adam-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()
model.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=10,callbacks=callbacks_list,batch_size=64) #eposh 8 batch 128 no relu hidden
model_json = model.to_json()
with open(datapath+"past-1-adam-model.json", "w") as json_file:
    json_file.write(model_json)

refreshed session
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 145, 300)          2023200   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 290)               517360    
_________________________________________________________________
dense_11 (Dense)             (None, 64)                18624     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 2,559,249
Trainable params: 536,049
Non-trainable params: 2,023,200
_________________________________________________________________
Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 18s 10ms/step - loss: 0.6387 - acc: 0.6334 - val_loss: 0.5910 - val_acc: 0.6892

Epoch 00001: val_acc improved from -inf to 0.68919, savi

Stacked BiLSTM for Past task

In [72]:
#Past LSTM2
session_refresh()
model2 = Sequential()
model2.add(Embedding(vocabulary_size,300, input_length = 145,weights = [embedding_matrix],trainable = False))
# model.add(Embedding(vocabulary_size, 145, input_length=145))
model2.add(Bidirectional(LSTM(145, dropout=0.2, recurrent_dropout=0.2,return_sequences=True)))
model2.add(Bidirectional(LSTM(145, dropout=0.1, recurrent_dropout=0.1)))
model2.add(Dense(64,activation = 'relu'))
model2.add(Dense(1, activation='sigmoid'))
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])
# model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
filepath=datapath+'/models/'+"past-2-best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model2.summary()


refreshed session
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 145, 300)          2023200   
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 145, 290)          517360    
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 290)               505760    
_________________________________________________________________
dense_19 (Dense)             (None, 64)                18624     
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 65        
Total params: 3,065,009
Trainable params: 1,041,809
Non-trainable params: 2,023,200
_________________________________________________________________


In [48]:
model2.fit(tr_data, tr_pt, validation_data=(val_data,val_pt), epochs=10,callbacks=callbacks_list,batch_size=64) #eposh 8 batch 128 no relu hidden
model2_json = model2.to_json()
with open(datapath+"past-2-model.json", "w") as json_file:
    json_file.write(model2_json)

Train on 1776 samples, validate on 592 samples
Epoch 1/10
1776/1776 [==============================] - 54s 31ms/step - loss: 0.6876 - acc: 0.6053 - val_loss: 0.6041 - val_acc: 0.6740

Epoch 00001: val_acc improved from -inf to 0.67399, saving model to gdrive/My Drive/data//models/past-2-best.hdf5
Epoch 2/10
1776/1776 [==============================] - 48s 27ms/step - loss: 0.6368 - acc: 0.6678 - val_loss: 0.5584 - val_acc: 0.7416

Epoch 00002: val_acc improved from 0.67399 to 0.74155, saving model to gdrive/My Drive/data//models/past-2-best.hdf5
Epoch 3/10
1776/1776 [==============================] - 50s 28ms/step - loss: 0.5412 - acc: 0.7545 - val_loss: 0.5322 - val_acc: 0.7568

Epoch 00003: val_acc improved from 0.74155 to 0.75676, saving model to gdrive/My Drive/data//models/past-2-best.hdf5
Epoch 4/10
1776/1776 [==============================] - 51s 29ms/step - loss: 0.4852 - acc: 0.8018 - val_loss: 0.5492 - val_acc: 0.7635

Epoch 00004: val_acc improved from 0.75676 to 0.76351, sa

In [75]:
model2.fit(np.concatenate((tr_data,val_data)), np.concatenate((tr_pt,val_pt)), epochs=9,batch_size=64)
model2_json = model2.to_json()
with open(datapath+"past-2-full-model.json", "w") as json_file:
    json_file.write(model2_json)
model2.save_weights(datapath+'/models/'+"past-2-full-best.hdf5")
print("Saved model to disk")

Epoch 1/1
2368/2368 [==============================] - 37s 15ms/step - loss: 0.3353 - acc: 0.8611
Saved model to disk


Evaluating the test data for future task 

In [80]:
scores = model2.evaluate(ts_data, ts_pt, verbose=0)

print("Accuracy: %.2f%%" % (scores[1]*100))



Accuracy: 78.75%


Accuracy: 78.75%

In [80]:
# testf=datapath+'sotu_for_analysis.tsv'
# df= pd.read_csv(testf,delimiter='\t',encoding='utf-8')
# full_st=np.array(df['sentence'].values)
# full_st = preprocess_sentences(full_st)

# #feature extraction
# #ts_st = sent_to_features(ts_st)
# sequences = tokenizer.texts_to_sequences(full_st)
# full_st_data = pad_sequences(sequences, maxlen=145)




y_pred = model.predict(full_st_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

391/391 [==============================] - 2s 5ms/step


In [0]:
df['pt']=list(map(int,y_pred))
df.to_csv('sotu_2019_analysis.tsv',sep='\t')

In [78]:
from keras.models import model_from_json
# load json and create model
json_file = open(datapath+"past-2-full-model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model2 = model_from_json(loaded_model_json)
# load weights into new model
model2.load_weights(datapath+'/models/'+"past-2-full-best.hdf5")
print("Loaded model from disk")
adg= optimizers.Adagrad(lr=0.01)
model2.compile(loss='binary_crossentropy', optimizer=adg, metrics=['accuracy'])

Loaded model from disk


Generating a classification report for the test data for future task

In [82]:
from sklearn.metrics import classification_report
y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
y_pred =(y_pred>0.5)

print(classification_report(ts_pt, y_pred))

593/593 [==============================] - 3s 5ms/step
              precision    recall  f1-score   support

         0.0       0.78      0.85      0.81       324
         1.0       0.80      0.71      0.75       269

   micro avg       0.79      0.79      0.79       593
   macro avg       0.79      0.78      0.78       593
weighted avg       0.79      0.79      0.79       593



Past Test Accuracy: 78.75% ; epochs 9, model: BiLSTM + Dense

precision    recall  f1-score   support

         0.0       0.78      0.85      0.81       324
         1.0       0.80      0.71      0.75       269

   micro avg       0.79      0.79      0.79       593
   
   macro avg       0.79      0.78      0.78       593
   
weighted avg       0.79      0.79      0.79       593


In [0]:
# scores = model2.evaluate(ts_data, ts_pt, verbose=0)

# print("Accuracy: %.2f%%" % (scores[1]*100))
# y_pred = model2.predict(ts_data, batch_size=64, verbose=1)
# y_pred =(y_pred>0.5)

# print(classification_report(ts_pt, y_pred))
